# Brick 1: Constants and Setup
Here we define the rules of the game.
- **0** means Cooperate.
- **1** means Defect.
- We also define the **Payoff Matrix** (the points we get).

In [9]:
!pip install gymnasium==1.1.1 moviepy==1.0.3

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/965.4 kB ? eta -:--:--
   ---------------------------------------- 0.0/965.4 kB ? eta -:--:--
   ---------------------------------------- 965.4/965.4 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: gymnasium
    Found existing installation: gymnasium 1.0.0
    Uninstalling gymnasium-1.0.0:
      Successfully uninstalled gymnasium-1.0.0



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: C:\Users\dorfe\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [10]:
import gymnasium as gym
import numpy as np
import random
from itertools import product


# 1. Define Actions for readability
COOPERATE = 0
DEFECT = 1

# 2. Define Rewards (Points for the Agent)
# Format: (Agent Action, Opponent Action): Reward
PAYOFF_MATRIX = {
    (COOPERATE, COOPERATE): 3, # Reward
    (COOPERATE, DEFECT):    0, # Sucker
    (DEFECT, COOPERATE):    5, # Temptation
    (DEFECT, DEFECT):       1, # Punishment
}

print("Setup Complete! Actions and Rewards defined.")

Setup Complete! Actions and Rewards defined.


# Brick 2: Opponent Strategies
This function decides the opponent's move based on the history of the game.
- **ALL_C**: Always nice.
- **ALL_D**: Always mean.
- **TFT (Tit-for-Tat)**: Copies your last move.
- **Imperfect TFT**: Copies your move 90% of the time, messes up 10% of the time.

In [11]:
def get_opponent_action(strategy, history):
    """
    Decides the opponent's move.
    history: A list of tuples, e.g., [(my_move, their_move), (my_move, their_move)]
    """

    # Strategy 1: Always Cooperate
    if strategy == "ALL_C":
        return COOPERATE

    # Strategy 2: Always Defect
    if strategy == "ALL_D":
        return DEFECT

    # Strategy 3: Tit-for-Tat (Copy my last move)
    if strategy == "TFT":
        if len(history) == 0:
            return COOPERATE # TFT starts nice

        # Look at the last round (index -1).
        # In history (my_move, opp_move), my_move is at index 0.
        my_last_move = history[-1][0]
        return my_last_move

    # Strategy 4: Imperfect Tit-for-Tat (10% chance of error)
    if strategy == "IMPERFECT_TFT":
        if len(history) == 0:
            return COOPERATE # Start nice

        # Calculate what standard TFT would do
        my_last_move = history[-1][0]
        intended_action = my_last_move

        # 10% chance to flip the action (Slip)
        if random.random() < 0.10:
            # Return the opposite (0 becomes 1, 1 becomes 0)
            return 1 - intended_action
        else:
            return intended_action

    return COOPERATE # Default fallback

# Brick 3: The Game Environment (The Class)
This class puts everything together.
- **__init__**: Sets up the game options (Opponent type, Memory length).
- **step**: Plays one round (Agent moves -> Opponent moves -> Calculate Score -> Update History).
- **reset**: Wipes the memory clean to start a new game.

In [12]:
class PrisonerDilemmaEnv(gym.Env):
    def __init__(self, opponent_strategy, memory_length=1):
        super().__init__()
        self.opponent_strategy = opponent_strategy
        self.memory_length = memory_length

        # Define Action Space (0 or 1)
        self.action_space = gym.spaces.Discrete(2)

        # Internal memory to track the game history
        self.history = []

    def reset(self, seed=None, options=None):
        """
        Resets the game to the starting state.
        The assignment says to assume everyone cooperated before the game started.
        """
        super().reset(seed=seed)
        self.history = [] # Clear history

        # Create the "Pre-game" history based on memory length
        # If memory is 1, we pretend the last round was (C, C)
        # If memory is 2, we pretend the last two rounds were (C, C), (C, C)
        for _ in range(self.memory_length):
            self.history.append((COOPERATE, COOPERATE))

        return self._get_state(), {}

    def step(self, action):
        """
        Plays one round of the game.
        """
        # 1. Agent makes a move (passed in as 'action')

        # 2. Opponent makes a move (using our helper function from Brick 2)
        opp_action = get_opponent_action(self.opponent_strategy, self.history)

        # 3. Calculate Reward (using the Matrix from Brick 1)
        reward = PAYOFF_MATRIX[(action, opp_action)]

        # 4. Update History
        # Add the new round to the end
        self.history.append((action, opp_action))

        # Remove the oldest round so we only keep what we need for memory
        if len(self.history) > self.memory_length:
            self.history.pop(0)

        # 5. Get the new State
        state = self._get_state()

        # In this specific assignment, the game repeats indefinitely (no "Game Over")
        # We handle the loop length in the experiment section.
        terminated = False
        truncated = False

        return state, reward, terminated, truncated, {}

    def _get_state(self):
        """
        Helper to format the current history as a Tuple (so it's easy to read).
        Example Memory-1: ((0, 0),)
        """
        return tuple(self.history)

print("Environment Class Defined Successfully!")

Environment Class Defined Successfully!


# Brick 4: Sanity Check
Let's play 5 rounds against a **Tit-for-Tat** opponent to see if it works.
We will play: C, D, D, C, C.
Since the opponent is TFT, they should copy our move from the *previous* turn.

In [13]:
# 1. Create the game environment
env = PrisonerDilemmaEnv(opponent_strategy="TFT", memory_length=1)

# 2. Reset to start
state, _ = env.reset()
print(f"Start State: {state}")

# 3. Play a few manual moves
my_moves = [COOPERATE, DEFECT, DEFECT, COOPERATE, COOPERATE]

for move in my_moves:
    # Take step
    state, reward, done, _, _ = env.step(move)

    # Translate numbers to words for printing
    move_name = "Cooperate" if move == 0 else "Defect"

    print(f"I played: {move_name} | New State: {state} | Reward: {reward}")

Start State: ((0, 0),)
I played: Cooperate | New State: ((0, 0),) | Reward: 3
I played: Defect | New State: ((1, 0),) | Reward: 5
I played: Defect | New State: ((1, 1),) | Reward: 1
I played: Cooperate | New State: ((0, 1),) | Reward: 0
I played: Cooperate | New State: ((0, 0),) | Reward: 3


# Brick 5A: Build the MDP Matrices

In [ ]:
def get_states(memory_length):
    """
    Generate all possible states for a given memory length.
    Memory-1: 4 states (your_move, opp_move)
    Memory-2: 16 states (your_move_t-1, your_move_t-2, opp_move_t-1, opp_move_t-2)
    """
    if memory_length == 1:
        # States are (my_last_move, opp_last_move)
        states = list(product([COOPERATE, DEFECT], repeat=2))
    elif memory_length == 2:
        # States are (my_t-1, my_t-2, opp_t-1, opp_t-2)
        states = list(product([COOPERATE, DEFECT], repeat=4))
    else:
        raise ValueError("Only memory_length 1 or 2 supported")

    return states


def build_transition_and_reward(opponent_strategy, memory_length):
    """
    Build the transition matrix P[s][a][s'] and reward matrix R[s][a].

    Returns:
        states: list of all states
        P: dict P[s][a] = dict of {s': probability}
        R: dict R[s][a] = expected immediate reward
    """
    states = get_states(memory_length)
    state_set = set(states)
    n_states = len(states)

    P = {s: {a: {} for a in ACTIONS} for s in states}
    R = {s: {a: 0.0 for a in ACTIONS} for s in states}

    for s in states:
        for a in ACTIONS:  # Agent's action

            if memory_length == 1:
                my_prev = s[0]  # My previous action (used by TFT)

                # Determine opponent's response distribution
                if opponent_strategy == "ALL_C":
                    opp_probs = {COOPERATE: 1.0}

                elif opponent_strategy == "ALL_D":
                    opp_probs = {DEFECT: 1.0}

                elif opponent_strategy == "TFT":
                    # Opponent copies my previous move
                    opp_probs = {my_prev: 1.0}

                elif opponent_strategy == "IMPERFECT_TFT":
                    # 90% copy, 10% opposite
                    opp_probs = {
                        my_prev: 0.9,
                        1 - my_prev: 0.1
                    }
                else:
                    raise ValueError(f"Unknown strategy: {opponent_strategy}")

                # Build transitions and expected rewards
                for opp_action, prob in opp_probs.items():
                    s_next = (a, opp_action)  # New state
                    if s_next in state_set:
                        P[s][a][s_next] = P[s][a].get(s_next, 0) + prob
                        R[s][a] += prob * PAYOFF_MATRIX[(a, opp_action)]

            elif memory_length == 2:
                # s = (my_t-1, my_t-2, opp_t-1, opp_t-2)
                my_t1, my_t2, opp_t1, opp_t2 = s

                # Opponent looks at my_t-1 (my most recent move in state)
                if opponent_strategy == "ALL_C":
                    opp_probs = {COOPERATE: 1.0}

                elif opponent_strategy == "ALL_D":
                    opp_probs = {DEFECT: 1.0}

                elif opponent_strategy == "TFT":
                    opp_probs = {my_t1: 1.0}

                elif opponent_strategy == "IMPERFECT_TFT":
                    opp_probs = {
                        my_t1: 0.9,
                        1 - my_t1: 0.1
                    }
                else:
                    raise ValueError(f"Unknown strategy: {opponent_strategy}")

                # Build transitions: s' = (a, my_t1, opp_new, opp_t1)
                for opp_action, prob in opp_probs.items():
                    s_next = (a, my_t1, opp_action, opp_t1)
                    if s_next in state_set:
                        P[s][a][s_next] = P[s][a].get(s_next, 0) + prob
                        R[s][a] += prob * PAYOFF_MATRIX[(a, opp_action)]

    return states, P, R

# Brick 5B: Policy Evaluation


In [ ]:
def policy_evaluation(states, P, R, policy, gamma, theta=1e-8):
    """
    Evaluate a policy by computing V^π(s) for all states.
    Uses iterative method until convergence.

    Args:
        states: list of states
        P: transition probabilities P[s][a][s']
        R: rewards R[s][a]
        policy: dict mapping state -> action
        gamma: discount factor
        theta: convergence threshold

    Returns:
        V: dict mapping state -> value
    """
    # Initialize V arbitrarily
    V = {s: 0.0 for s in states}

    iteration = 0
    while True:
        delta = 0
        for s in states:
            v_old = V[s]
            a = policy[s]

            # V(s) = R(s,a) + gamma * sum_{s'} P(s'|s,a) * V(s')
            v_new = R[s][a]
            for s_next, prob in P[s][a].items():
                v_new += gamma * prob * V[s_next]

            V[s] = v_new
            delta = max(delta, abs(v_old - v_new))

        iteration += 1
        if delta < theta:
            break

    return V, iteration


# Brick 5C: Policy Improvement


In [ ]:
def policy_improvement(states, P, R, V, gamma):
    """
    Improve the policy by acting greedily with respect to V.

    Returns:
        new_policy: dict mapping state -> action
        policy_stable: bool, True if policy didn't change
    """
    new_policy = {}

    for s in states:
        best_action = None
        best_value = float('-inf')

        for a in ACTIONS:
            # Q(s,a) = R(s,a) + gamma * sum_{s'} P(s'|s,a) * V(s')
            q_value = R[s][a]
            for s_next, prob in P[s][a].items():
                q_value += gamma * prob * V[s_next]

            if q_value > best_value:
                best_value = q_value
                best_action = a

        new_policy[s] = best_action

    return new_policy


# Brick 5D: Policy Iteration (Main Algorithm)


In [ ]:
def policy_iteration(opponent_strategy, memory_length, gamma, verbose=True):
    """
    Run Policy Iteration to find the optimal policy.

    Args:
        opponent_strategy: "ALL_C", "ALL_D", "TFT", or "IMPERFECT_TFT"
        memory_length: 1 or 2
        gamma: discount factor
        verbose: print progress

    Returns:
        optimal_policy: dict mapping state -> action
        V: value function
        history: list of (policy, V, eval_iterations) per iteration
    """
    # Build MDP
    states, P, R = build_transition_and_reward(opponent_strategy, memory_length)

    # Initialize policy (start with all cooperate)
    policy = {s: COOPERATE for s in states}

    history = []
    iteration = 0

    while True:
        # Policy Evaluation
        V, eval_iters = policy_evaluation(states, P, R, policy, gamma)

        # Policy Improvement
        new_policy = policy_improvement(states, P, R, V, gamma)

        # Check for convergence
        policy_stable = all(policy[s] == new_policy[s] for s in states)

        history.append({
            'iteration': iteration,
            'policy': policy.copy(),
            'V': V.copy(),
            'eval_iterations': eval_iters
        })

        if verbose:
            print(f"Iteration {iteration}: Policy Evaluation took {eval_iters} iterations")
            print(f"  Policy: {format_policy(policy, memory_length)}")

        if policy_stable:
            if verbose:
                print(f"\n✓ Policy converged after {iteration + 1} iterations!")
            break

        policy = new_policy
        iteration += 1

        # Safety limit
        if iteration > 100:
            print("Warning: Max iterations reached")
            break

    return policy, V, history


# Brick 5E: Helper Functions


In [ ]:
def format_policy(policy, memory_length):
    """Pretty print the policy."""
    action_names = {COOPERATE: 'C', DEFECT: 'D'}

    if memory_length == 1:
        result = {}
        for s, a in policy.items():
            s_str = f"({action_names[s[0]]},{action_names[s[1]]})"
            result[s_str] = action_names[a]
        return result
    else:
        # For Memory-2, just show a summary
        return {str(s): action_names[a] for s, a in policy.items()}


def format_state(s, memory_length):
    """Format a state tuple nicely."""
    names = {COOPERATE: 'C', DEFECT: 'D'}
    return tuple(names[x] for x in s)


# Brick 6: Simulation Verification


In [ ]:
def simulate_policy(opponent_strategy, memory_length, policy, n_episodes=50, n_steps=50, seed=42):
    """
    Simulate the optimal policy against the opponent.

    Returns:
        avg_cumulative_reward: average total reward per episode
        all_rewards: list of episode rewards
    """
    np.random.seed(seed)
    all_rewards = []

    for episode in range(n_episodes):
        # Initialize state (all cooperate history)
        if memory_length == 1:
            state = (COOPERATE, COOPERATE)
            history = [(COOPERATE, COOPERATE)]
        else:
            state = (COOPERATE, COOPERATE, COOPERATE, COOPERATE)
            history = [(COOPERATE, COOPERATE), (COOPERATE, COOPERATE)]

        episode_reward = 0

        for step in range(n_steps):
            # Agent acts according to policy
            action = policy[state]

            # Opponent responds
            if memory_length == 1:
                my_prev = state[0]
            else:
                my_prev = state[0]  # my_t-1

            if opponent_strategy == "ALL_C":
                opp_action = COOPERATE
            elif opponent_strategy == "ALL_D":
                opp_action = DEFECT
            elif opponent_strategy == "TFT":
                opp_action = my_prev
            elif opponent_strategy == "IMPERFECT_TFT":
                if np.random.random() < 0.9:
                    opp_action = my_prev
                else:
                    opp_action = 1 - my_prev

            # Get reward
            reward = PAYOFF_MATRIX[(action, opp_action)]
            episode_reward += reward

            # Update state
            if memory_length == 1:
                state = (action, opp_action)
            else:
                state = (action, state[0], opp_action, state[2])

        all_rewards.append(episode_reward)

    return np.mean(all_rewards), all_rewards




# Main: Run All Experiments


In [ ]:
if __name__ == "__main__":
    opponents = ["ALL_C", "ALL_D", "TFT", "IMPERFECT_TFT"]
    gammas = [0.1, 0.5, 0.9, 0.99]
    memory_lengths = [1, 2]

    print("=" * 60)
    print("POLICY ITERATION EXPERIMENTS")
    print("=" * 60)

    results = {}

    for memory in memory_lengths:
        print(f"\n{'='*60}")
        print(f"MEMORY-{memory}")
        print(f"{'='*60}")

        for opponent in opponents:
            print(f"\n--- Opponent: {opponent} ---")

            for gamma in gammas:
                print(f"\nγ = {gamma}")

                # Run Policy Iteration
                policy, V, history = policy_iteration(
                    opponent_strategy=opponent,
                    memory_length=memory,
                    gamma=gamma,
                    verbose=False
                )

                # Print optimal policy
                print(f"  Optimal Policy: {format_policy(policy, memory)}")

                # Simulate to verify
                avg_reward, _ = simulate_policy(
                    opponent_strategy=opponent,
                    memory_length=memory,
                    policy=policy,
                    n_episodes=50,
                    n_steps=50
                )
                print(f"  Avg Cumulative Reward (50 episodes × 50 steps): {avg_reward:.2f}")

                # Store results
                key = (memory, opponent, gamma)
                results[key] = {
                    'policy': policy,
                    'V': V,
                    'iterations': len(history),
                    'avg_reward': avg_reward
                }

    print("\n" + "=" * 60)
    print("SUMMARY TABLE")
    print("=" * 60)

    # Print summary
    for memory in memory_lengths:
        print(f"\n=== Memory-{memory} ===")
        print(f"{'Opponent':<15} {'γ=0.1':<12} {'γ=0.5':<12} {'γ=0.9':<12} {'γ=0.99':<12}")
        print("-" * 55)

        for opponent in opponents:
            row = f"{opponent:<15}"
            for gamma in gammas:
                key = (memory, opponent, gamma)
                policy = results[key]['policy']
                # Count how many states have Defect
                defects = sum(1 for a in policy.values() if a == DEFECT)
                total = len(policy)
                row += f" {defects}/{total} D".ljust(12)
            print(row)


POLICY ITERATION EXPERIMENTS

MEMORY-1

--- Opponent: ALL_C ---

γ = 0.1


NameError: name 'ACTIONS' is not defined